In [1]:
import sys
print(sys.version)

import sysconfig
print(sysconfig.get_paths()['purelib'])

3.10.9 | packaged by conda-forge | (main, Jan 11 2023, 15:15:40) [MSC v.1916 64 bit (AMD64)]
D:\olaf\miniconda3_py310\Lib\site-packages


In [2]:
import esa_snappy
import snapista

from snapista.binning import aggregators
from snapista.binning import BinningVariable, BinningOutputBands, BinningBand, Aggregators, BinningVariables
from snapista import Graph
from snapista import Operator
import lxml.etree as etree
import os

## Create an Aggregator

In [3]:
ag = aggregators.AggregatorAvg(varName='IWV', targetName="iwv", outputCounts='true', outputSums='true', weightCoeff=1.0)

## Create a binning operator

In [4]:
binning = Operator('Binning')

binning.sourceProductFormat = 'sen3'

binning.aggregatorConfigs = Aggregators([ag,
                                        aggregators.AggregatorAvgOutlier(varName='IWV', targetName='outlier'),
                                         aggregators.AggregatorMinMax(varName='IWV', targetName='iwv'),
                                        aggregators.AggregatorOnMaxSet(targetName='omvn', onMaxVarName='IWV', setVarNames='IWV'),
                                        aggregators.AggregatorPercentile(varName='IWV', percentage=90, targetName='percent'),
                                        aggregators.AggregatorSum(varName='IWV', targetName='sum')])

binning.sourceProductPaths = './data/S3*/*.xml'

binning.outputFile = './data/IWV_binning_result.dim'

In [5]:
binning

Operator('Binning', sourceProductPaths='./data/S3*/*.xml', sourceProductFormat='sen3', sourceGraphPaths='None', region='None', startDateTime='None', periodDuration='None', timeFilterMethod='NONE', minDataHour='None', numRows='2160', superSampling='1', maxDistanceOnEarth='-1', maskExpr='None', variableConfigs='None', aggregatorConfigs='<snapista.binning.aggregators.Aggregators object at 0x00000110BE268550>', postProcessorConfig='None', outputType='Product', outputFile='./data/IWV_binning_result.dim', outputFormat='BEAM-DIMAP', bandConfigurations='None', productCustomizerConfig='None', outputBinnedData='false', outputTargetProduct='true', metadataPropertiesFile='./metadata.properties', metadataTemplateDir='.', metadataAggregatorName='NAME', planetaryGridClass='org.esa.snap.binning.support.SEAGrid')

## Create the Graph

In [6]:
g = Graph()

g.add_node(operator=binning, 
           node_id='binning_1')

aggregatorConfigs <snapista.binning.aggregators.Aggregators object at 0x00000110BE268550>
Instance TargetBandDescriptors: False
Instance Aggregators: True
Instance BinningOutputBands: False
Instance BinningVariables: False
Instance str: False
bandConfigurations None
postProcessorConfig None
productCustomizerConfig None
variableConfigs None


In [7]:
g.add_node(operator=binning, 
           node_id='binning_1')

```xml
<aggregators>
        <aggregator>
            <type>AVG</type>
            <varName>Rw665_rep</varName>
            <targetName>avg</targetName>
            <weightCoeff>1.0</weightCoeff>
            <outputCounts>true</outputCounts>
            <outputSums>true</outputSums>
        </aggregator>
        <aggregator>
            <type>MIN_MAX</type>
            <varName>Rw665_rep</varName>
            <targetName>Rw665_rep</targetName>
        </aggregator>
        <aggregator>
            <type>AVG_OUTLIER</type>
            <varName>Rw665_rep</varName>
            <targetName/>
        </aggregator>
        <aggregator>
            <type>SUM</type>
            <varName>Rw665_rep</varName>
            <targetName>aaa</targetName>
        </aggregator>
        <aggregator>
            <type>PERCENTILE</type>
            <varName>Rw665_rep</varName>
        </aggregator>
        <aggregator>
            <type>ON_MAX_SET</type>
            <onMaxVarName>Rw665_rep</onMaxVarName>
            <setVarNames>Rw665_rep</setVarNames>
        </aggregator>
    </aggregators>
```

In [8]:
g.view()

<graph>
  <version>1.0</version>
  <node id="binning_1">
    <operator>Binning</operator>
    <sources/>
    <parameters class="com.bc.ceres.binding.dom.XppDomElement">
      <aggregators>
        <aggregator>
          <type>AVG</type>
          <varName>IWV</varName>
          <targetName>iwv</targetName>
          <weightCoeff>1.0</weightCoeff>
          <outputCounts>true</outputCounts>
          <outputSums>true</outputSums>
        </aggregator>
        <aggregator>
          <type>AVG_OUTLIER</type>
          <varName>IWV</varName>
          <targetName>outlier</targetName>
        </aggregator>
        <aggregator>
          <type>MIN_MAX</type>
          <varName>IWV</varName>
          <targetName>iwv</targetName>
        </aggregator>
        <aggregator>
          <type>ON_MAX_SET</type>
          <targetName>omvn</targetName>
          <onMaxVarName>IWV</onMaxVarName>
          <setVarNames>IWV</setVarNames>
        </aggregator>
        <aggregator>
          <type>PERCEN

In [9]:
g.save_graph(filename='./data/c_gls_binning.xml')

#g.run(gpt_options=["--diag", "-e","-x", "-c", "1024M"])
g.run()

Processing the graph, this may take a while. Please wait...
standard output of subprocess: 
Executing processing graph
.13%.27%.41%.55%.69%.83%. done.
standard error of subprocess: 
SEVERE: org.esa.snap.runtime.Engine: Failed to modify class loader field 'usr_paths'
java.lang.NoSuchMethodException: java.lang.ClassLoader.initializePath(java.lang.String)
	at java.base/java.lang.Class.getDeclaredMethod(Class.java:2848)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:286)
	at org.esa.snap.runtime.Engine.setJavaLibraryPath(Engine.java:265)
	at org.esa.snap.runtime.Engine.<init>(Engine.java:46)
	at org.esa.snap.runtime.Engine.start(Engine.java:117)
	at org.esa.snap.runtime.Engine.start(Engine.java:90)
	at org.esa.snap.runtime.Launcher.run(Launcher.java:51)
	at org.esa.snap.runtime.Launcher.main(Launcher.java:31)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580

0